You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [27]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [28]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [29]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [30]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [31]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [32]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [33]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [34]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [35]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [39]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-07-30 09:39:53,509 - 140223640587136 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [41]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: Unveiling the Power of Artificial Intelligence: Latest Trends and Key Players

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's world
- Brief overview of the latest trends and key players in the AI industry

II. Latest Trends in Artificial Intelligence
A. Machine Learning
B. Natural Language Processing
C. Robotics
D. AI in Healthcare
E. AI in Finance

III. Key Players in the AI Industry
A. Google
B. Amazon
C. Microsoft
D. IBM
E. Fac

I now can give a great answer

Final Answer:
# Unveiling the Power of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become a ubiquitous term in today's world, encompassing a wide range of technologies that aim to replicate human intelligence in machines. From self-driving cars to virtual assistants, AI is transforming various industries and revolutionizing the way we live and work. In this blog post, we will delve into the latest trends and key players in the AI industry, shedding light on the advancements that are shaping our future.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine learning, a subset of AI, enables machines to learn from data and improve their performance without being explicitly programmed. This technology powers recommendation systems, autonomous vehicles, and predictive analytics, making it a cornerstone of AI innovation.

### Natural Language Processing
Natural Language Processin

- Display the results of your execution as markdown in the notebook.

In [42]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Power of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become a ubiquitous term in today's world, encompassing a wide range of technologies that aim to replicate human intelligence in machines. From self-driving cars to virtual assistants, AI is transforming various industries and revolutionizing the way we live and work. In this blog post, we will delve into the latest trends and key players in the AI industry, shedding light on the advancements that are shaping our future.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine learning, a subset of AI, enables machines to learn from data and improve their performance without being explicitly programmed. This technology powers recommendation systems, autonomous vehicles, and predictive analytics, making it a cornerstone of AI innovation.

### Natural Language Processing
Natural Language Processing (NLP) focuses on enabling computers to understand, interpret, and generate human language. From chatbots to language translation tools, NLP is enhancing communication between humans and machines, opening up new possibilities for AI applications.

### Robotics
AI-driven robotics is revolutionizing industries such as manufacturing, healthcare, and logistics. Collaborative robots, or cobots, are working alongside humans in factories, while surgical robots are assisting surgeons in complex procedures, showcasing the potential of AI-powered robotics.

### AI in Healthcare
The healthcare industry is leveraging AI to enhance patient care, streamline operations, and accelerate medical research. AI-powered diagnostic tools, personalized treatment plans, and predictive analytics are revolutionizing healthcare delivery and improving patient outcomes.

### AI in Finance
In the finance sector, AI is being used to detect fraud, automate trading, and personalize customer experiences. Robo-advisors, algorithmic trading systems, and chatbots are reshaping the financial services landscape, offering efficiency and convenience to consumers and businesses alike.

## Key Players in the AI Industry

### Google
Google's deep investments in AI research and development have led to breakthroughs in machine learning, NLP, and computer vision. From Google Assistant to self-driving cars, Google is at the forefront of AI innovation, pushing the boundaries of what is possible with artificial intelligence.

### Amazon
Amazon's AI capabilities power its e-commerce platform, logistics operations, and cloud services. With advancements in recommendation systems, voice recognition, and drone technology, Amazon is redefining customer experiences and operational efficiency through AI-driven solutions.

### Microsoft
Microsoft's Azure AI platform offers a comprehensive suite of AI tools and services, enabling businesses to build and deploy AI applications at scale. From cognitive services to conversational AI, Microsoft is empowering organizations to harness the power of AI for digital transformation and innovation.

### IBM
IBM's Watson AI platform is a pioneer in cognitive computing, with applications in healthcare, finance, and cybersecurity. IBM's AI research and solutions are driving advancements in natural language processing, computer vision, and decision-making systems, shaping the future of AI-enabled technologies.

### Facebook
Facebook's AI initiatives focus on improving user experiences, content moderation, and virtual reality applications. With AI-powered recommendation algorithms, language translation tools, and facial recognition technology, Facebook is leveraging AI to connect people and drive social interactions on its platform.

## Target Audience Analysis

Professionals in the tech industry, business owners, students, researchers, and curious consumers can all benefit from staying informed about AI trends and advancements. By understanding the latest developments in AI and the key players driving innovation, individuals can explore the potential applications of AI in their respective fields and stay ahead of the curve in this rapidly evolving industry.

## Call to Action

As we continue to witness the evolution of AI technologies, it is essential for readers to stay updated on AI trends and advancements. I encourage you to explore the diverse applications of AI in various industries, from healthcare to finance, and to discover how AI can transform your business or research endeavors. For further reading and research, check out the recommended resources below and delve deeper into the fascinating world of Artificial Intelligence.

In conclusion, Artificial Intelligence is reshaping the future of technology and society, with a myriad of applications and innovations that are set to revolutionize our world. By staying informed and engaged with the latest trends and key players in the AI industry, we can harness the power of AI to drive progress and create a more intelligent future for all.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [44]:
topic = "Capybara take over the world"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Capybara take over the world.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document:

Title: Capybara Take Over the World: A Look at the Latest Trends and Key Players

Introduction:
- Brief overview of the increasing population and influence of capybaras worldwide
- Mention of the potential implications of capybaras taking over the world
- Preview of key points to be discussed in the article

Key Points:
1. Latest Trends:
- Growth of capybara populations in various regions around the world
- Increase in media coverage and social media presence of capybaras
- Adoptio

I now can give a great answer

Final Answer:
# Capybara Take Over the World: A Look at the Latest Trends and Key Players

## Introduction
Capybaras, the world's largest rodents, have been making waves in the global wildlife scene with their increasing population and influence. From the savannas of South America to unexpected sightings in urban areas, these charismatic creatures are capturing the hearts of animal lovers and conservationists worldwide. As their numbers grow, the potential implications of capybaras taking over the world raise questions about the delicate balance of ecosystems and human-wildlife interactions. In this article, we delve into the latest trends and key players in the capybara phenomenon, shedding light on their impact and the challenges they face in a changing world.

## Latest Trends
The growth of capybara populations in various regions around the world has been a notable trend in recent years. Originally native to South America, these adaptable creatures hav

In [45]:
Markdown(result)

# Capybara Take Over the World: A Look at the Latest Trends and Key Players

## Introduction
Capybaras, the world's largest rodents, have been making waves in the global wildlife scene with their increasing population and influence. From the savannas of South America to unexpected sightings in urban areas, these charismatic creatures are capturing the hearts of animal lovers and conservationists worldwide. As their numbers grow, the potential implications of capybaras taking over the world raise questions about the delicate balance of ecosystems and human-wildlife interactions. In this article, we delve into the latest trends and key players in the capybara phenomenon, shedding light on their impact and the challenges they face in a changing world.

## Latest Trends
The growth of capybara populations in various regions around the world has been a notable trend in recent years. Originally native to South America, these adaptable creatures have found new habitats in places like Japan and parts of the United States, leading to both excitement and concern among wildlife enthusiasts. Alongside this expansion, there has been a surge in media coverage and social media presence of capybaras, with viral videos and adorable photos showcasing their playful nature and unique behaviors. Additionally, some countries have seen a rise in the adoption of capybaras as pets, blurring the lines between wild and domesticated animals.

## Key Players
Capybara conservation organizations and researchers play a crucial role in understanding and protecting these gentle giants. By studying their behavior and habitat requirements, experts aim to ensure the long-term survival of capybara populations in the face of environmental threats and human activities. Social media influencers and popular websites featuring capybaras have also contributed to raising awareness about these fascinating creatures, sparking interest and curiosity among a diverse audience. Moreover, countries with the largest capybara populations, such as Brazil and Venezuela, are at the forefront of efforts to conserve and manage these iconic species for future generations to enjoy.

## Noteworthy News
Recent studies on capybara behavior and adaptability have provided valuable insights into their social structures and ecological roles. Researchers have observed instances of capybaras interacting with other wildlife species, highlighting their importance as keystone species in their ecosystems. Updates on laws and regulations regarding the ownership and protection of capybaras reflect ongoing efforts to balance human needs with wildlife conservation priorities. As the world grapples with environmental challenges, the fate of capybaras serves as a reminder of the interconnectedness of all living beings and the shared responsibility to protect biodiversity.

In conclusion, the rise of capybaras on the global stage presents a unique opportunity to appreciate and conserve these remarkable animals. By staying informed about the latest trends, supporting key players in capybara conservation, and staying engaged with noteworthy news, we can contribute to a sustainable future where capybaras and other wildlife thrive. Let us embrace the capybara takeover with curiosity and compassion, knowing that our actions today will shape the world they inherit tomorrow. 

*This opinion piece aims to provide a balanced perspective on the topic of capybaras taking over the world, incorporating insights from the Content Planner's outline and additional research on capybara behavior and conservation efforts.*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).